In [1]:
from purpleair.network import SensorList
import pandas as pd
import numpy as np
import time
import datetime
from datetime import date

In [2]:
p = SensorList()  # Initialized 11,220 sensors!
# Other sensor filters include 'outside', 'useful', 'family', and 'no_child'
df = p.to_dataframe(sensor_filter='all',
                    channel='parent')

Initialized 22,520 sensors!


Warnow-Chacko(South-west): 10894
Urbana(South-east):94415
Urbana on Holmes:107592

In [3]:
from purpleair.sensor import Sensor
WarnowChacko = Sensor(10894, parse_location=True)
print(WarnowChacko)
Urbana = Sensor(94415, parse_location=True)
print(Urbana)
UrbanaOnHolmes = Sensor(107592, parse_location=True)
print(UrbanaOnHolmes)

Sensor 10894 at 1778, Foxborough Court, Champaign, Champaign County, Illinois, 61822, United States
Sensor 94415 at 1559, South Vine Street, West Urbana, Urbana, Champaign County, Illinois, 61801, United States
Sensor 107592 at 345, East Holmes Street, Urbana, Champaign County, Illinois, 61801, United States


In [4]:
df_WarnowChacko = WarnowChacko.parent.get_historical(weeks_to_get= 14, start_date = date(2021, 12, 7),
                              thingspeak_field='primary')

In [5]:
df_Urbana = Urbana.parent.get_historical(weeks_to_get= 14, start_date = date(2021, 12, 7),
                              thingspeak_field='primary')

In [6]:
df_UrbanaOnHolmes = UrbanaOnHolmes.parent.get_historical(weeks_to_get= 14, start_date = date(2021, 12, 7),
                              thingspeak_field='primary')

In [7]:
df_WarnowChacko.created_at=pd.DatetimeIndex(df_WarnowChacko.created_at).tz_convert('US/Central')
df_WarnowChacko['created_at'] = df_WarnowChacko['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S').tolist()

df_Urbana.created_at=pd.DatetimeIndex(df_Urbana.created_at).tz_convert('US/Central')
df_Urbana['created_at'] = df_Urbana['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S').tolist()

df_UrbanaOnHolmes.created_at=pd.DatetimeIndex(df_UrbanaOnHolmes.created_at).tz_convert('US/Central')
df_UrbanaOnHolmes['created_at'] = df_UrbanaOnHolmes['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S').tolist()

In [10]:
# Read feather file
df3 = pd.read_feather('export_UIUC_feather', columns=None, use_threads=True)
df3 = df3.drop(['index'], axis=1)
df3

,reading_datestamp,co_prescaled,CO2_prescaled,no_prescaled,no2_prescaled,o3_prescaled,pm1_prescale,pm2_5_prescale,pm4_prescale,pm10_prescale,pm_total_prescale,pm_tpc_prescale,nox_prescaled,temperature_f,pressure,humidity
0,2021-09-02 04:45:00,250.38,420.35,15.87,6.86,14.86,5.38,6.94,8.00,10.23,10.23,2.91,13.14,75.9,992.6,63.6
1,2021-09-02 05:00:00,255.19,418.04,15.78,7.10,16.61,5.26,6.89,7.97,9.76,9.76,2.82,12.58,77.0,992.5,62.5
2,2021-09-02 05:15:00,254.69,413.88,15.90,7.30,18.42,5.35,6.98,8.12,11.68,11.70,2.88,14.58,77.7,993.0,61.5
3,2021-09-02 05:30:00,265.17,412.49,15.54,7.48,20.59,5.20,6.77,7.84,9.82,9.82,2.78,12.60,78.4,992.3,61.0
4,2021-09-02 05:45:00,256.95,408.79,17.10,6.56,23.36,5.40,6.97,8.12,10.32,10.32,2.92,13.24,79.2,992.9,59.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8540,2021-11-30 14:00:00,300.71,776.00,3.34,5.50,30.32,3.20,4.52,5.50,9.11,9.11,1.83,8.84,54.3,992.7,58.5
8541,2021-11-30 14:15:00,303.22,775.16,2.44,5.20,29.64,3.21,4.38,5.31,9.70,9.72,1.81,7.64,54.1,992.9,58.8
8542,2021-11-30 14:30:00,313.67,776.00,3.17,5.35,29.84,3.20,4.41,5.38,9.14,9.14,1.82,8.52,54.0,992.9,58.8
8543,2021-11-30 14:45:00,309.89,776.85,5.20,4.53,33.18,3.15,4.51,5.44,8.46,8.46,1.79,9.73,53.6,992.9,58.8


In [11]:
def each_fifteen(time_str):
    new_time = datetime.datetime.strptime(time_str,'%Y-%m-%d %H:%M:%S')
    new_time = new_time.replace(second = 0)
    if new_time.minute < 8:
        new_time = new_time.replace(minute = 0)
    elif new_time.minute < 23:
        new_time = new_time.replace(minute = 15)
    elif new_time.minute < 38:
        new_time = new_time.replace(minute = 30)
    elif new_time.minute < 53:
        new_time = new_time.replace(minute = 45)
    elif new_time.minute < 60:
        new_time = new_time + datetime.timedelta(hours = 1)
        new_time = new_time.replace(minute = 0)
    return str(new_time)

In [13]:
# df_WarnowChacko.apply(each_fifteen, axis = 0)
df_WarnowChacko['new_date'] = df_WarnowChacko.apply(lambda time: each_fifteen(time['created_at']),axis=1)
df_WarnowChacko

,created_at,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3,new_date
entry_id,,,,,,,,,,
978053,2021-11-29 18:01:19,2.00,3.57,3.73,18121.0,-68.0,47.0,38.0,3.57,2021-11-29 18:00:00
978054,2021-11-29 18:03:19,2.10,3.81,4.66,18123.0,-66.0,47.0,38.0,3.81,2021-11-29 18:00:00
978055,2021-11-29 18:05:19,2.31,3.81,3.81,18125.0,-66.0,47.0,38.0,3.81,2021-11-29 18:00:00
978056,2021-11-29 18:07:19,2.36,4.11,4.18,18127.0,-67.0,47.0,39.0,4.11,2021-11-29 18:00:00
978057,2021-11-29 18:09:19,1.40,2.96,3.70,18129.0,-66.0,47.0,39.0,2.96,2021-11-29 18:15:00
...,...,...,...,...,...,...,...,...,...,...
931261,2021-08-30 17:36:56,5.02,7.78,8.00,8651.0,-61.0,87.0,51.0,7.78,2021-08-30 17:30:00
931262,2021-08-30 17:38:56,5.14,6.71,6.71,8653.0,-61.0,87.0,51.0,6.71,2021-08-30 17:45:00
931263,2021-08-30 17:40:56,4.70,6.34,6.34,8655.0,-61.0,87.0,51.0,6.34,2021-08-30 17:45:00


In [14]:
df_Urbana['new_date'] = df_Urbana.apply(lambda time: each_fifteen(time['created_at']),axis=1)
df_Urbana

,created_at,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3,new_date
entry_id,,,,,,,,,,
194084,2021-11-29 18:00:45,6.02,6.93,7.28,316.0,-58.0,44.0,45.0,6.93,2021-11-29 18:00:00
194085,2021-11-29 18:02:45,6.86,7.68,7.70,318.0,-58.0,44.0,46.0,7.68,2021-11-29 18:00:00
194086,2021-11-29 18:04:45,6.68,7.43,7.91,320.0,-59.0,44.0,46.0,7.43,2021-11-29 18:00:00
194087,2021-11-29 18:06:45,6.56,7.88,8.27,322.0,-59.0,44.0,46.0,7.88,2021-11-29 18:00:00
194088,2021-11-29 18:08:45,6.79,8.16,8.83,324.0,-59.0,44.0,46.0,8.16,2021-11-29 18:15:00
...,...,...,...,...,...,...,...,...,...,...
128716,2021-08-30 18:50:11,3.37,4.39,4.59,15779.0,-58.0,86.0,54.0,4.39,2021-08-30 18:45:00
128717,2021-08-30 18:52:11,3.45,4.34,4.34,15781.0,-56.0,86.0,54.0,4.34,2021-08-30 18:45:00
128718,2021-08-30 18:54:11,4.00,5.05,5.57,15783.0,-55.0,85.0,55.0,5.05,2021-08-30 19:00:00


In [16]:
df_UrbanaOnHolmes['new_date'] = df_UrbanaOnHolmes.apply(lambda time: each_fifteen(time['created_at']),axis=1)
df_UrbanaOnHolmes

,created_at,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3,new_date
entry_id,,,,,,,,,,
90723,2021-11-22 18:01:31,2.14,3.37,3.37,47487.0,-64.0,38.0,34.0,3.37,2021-11-22 18:00:00
90724,2021-11-22 18:03:31,1.74,2.61,3.19,47489.0,-65.0,38.0,34.0,2.61,2021-11-22 18:00:00
90725,2021-11-22 18:05:31,1.55,3.19,4.16,47491.0,-66.0,38.0,34.0,3.19,2021-11-22 18:00:00
90726,2021-11-22 18:07:31,1.43,2.36,2.38,47493.0,-64.0,38.0,34.0,2.36,2021-11-22 18:00:00
90727,2021-11-22 18:09:31,1.44,2.81,2.81,47495.0,-65.0,38.0,34.0,2.81,2021-11-22 18:15:00
...,...,...,...,...,...,...,...,...,...,...
30277,2021-08-30 18:50:16,2.69,4.24,4.24,31567.0,-65.0,83.0,59.0,4.24,2021-08-30 18:45:00
30278,2021-08-30 18:52:16,2.60,4.14,4.14,31569.0,-64.0,83.0,60.0,4.14,2021-08-30 18:45:00
30279,2021-08-30 18:54:16,2.62,4.14,4.30,31571.0,-66.0,83.0,59.0,4.14,2021-08-30 19:00:00


In [17]:
df_WarnowChacko_date = df_WarnowChacko.drop(columns = ['created_at'])
df_WarnowChacko_date.sort_values('new_date', ascending=True)

,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3,new_date
entry_id,,,,,,,,,
926266,11.79,18.14,19.02,7673.0,-65.0,94.0,45.0,18.14,2021-08-23 19:00:00
926265,12.02,17.07,18.05,7671.0,-65.0,94.0,45.0,17.07,2021-08-23 19:00:00
926267,11.85,17.62,18.69,7675.0,-69.0,94.0,45.0,17.62,2021-08-23 19:00:00
926264,12.54,19.31,19.76,7669.0,-71.0,94.0,45.0,19.31,2021-08-23 19:00:00
926268,11.72,17.15,18.07,7677.0,-67.0,94.0,45.0,17.15,2021-08-23 19:15:00
...,...,...,...,...,...,...,...,...,...
979869,2.41,3.93,4.66,21753.0,-68.0,53.0,51.0,3.93,2021-12-02 06:30:00
979870,2.34,3.82,4.34,21755.0,-67.0,53.0,51.0,3.82,2021-12-02 06:30:00
979871,2.18,3.75,3.75,21757.0,-67.0,50.0,51.0,3.75,2021-12-02 06:30:00


In [18]:
df_Urbana_date = df_Urbana.drop(columns = ['created_at'])
df_Urbana_date.sort_values('new_date', ascending=True)

,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3,new_date
entry_id,,,,,,,,,
123682,12.32,15.88,16.96,5709.0,-60.0,94.0,45.0,15.88,2021-08-23 19:00:00
123683,12.93,18.24,18.83,5711.0,-56.0,94.0,44.0,18.24,2021-08-23 19:00:00
123684,12.15,16.14,16.46,5713.0,-59.0,93.0,44.0,16.14,2021-08-23 19:00:00
123685,12.57,15.69,16.34,5715.0,-58.0,93.0,45.0,15.69,2021-08-23 19:00:00
123686,12.87,17.09,17.65,5717.0,-58.0,93.0,43.0,17.09,2021-08-23 19:15:00
...,...,...,...,...,...,...,...,...,...
195901,3.40,5.02,5.31,3950.0,-56.0,49.0,60.0,5.02,2021-12-02 06:30:00
195902,3.40,4.68,5.46,3952.0,-54.0,49.0,60.0,4.68,2021-12-02 06:30:00
195905,3.43,4.17,4.81,3958.0,-54.0,49.0,60.0,4.17,2021-12-02 06:45:00


In [19]:
df_UrbanaOnHolmes_date = df_Urbana.drop(columns = ['created_at'])
df_UrbanaOnHolmes_date.sort_values('new_date', ascending=True)

,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3,new_date
entry_id,,,,,,,,,
123682,12.32,15.88,16.96,5709.0,-60.0,94.0,45.0,15.88,2021-08-23 19:00:00
123683,12.93,18.24,18.83,5711.0,-56.0,94.0,44.0,18.24,2021-08-23 19:00:00
123684,12.15,16.14,16.46,5713.0,-59.0,93.0,44.0,16.14,2021-08-23 19:00:00
123685,12.57,15.69,16.34,5715.0,-58.0,93.0,45.0,15.69,2021-08-23 19:00:00
123686,12.87,17.09,17.65,5717.0,-58.0,93.0,43.0,17.09,2021-08-23 19:15:00
...,...,...,...,...,...,...,...,...,...
195901,3.40,5.02,5.31,3950.0,-56.0,49.0,60.0,5.02,2021-12-02 06:30:00
195902,3.40,4.68,5.46,3952.0,-54.0,49.0,60.0,4.68,2021-12-02 06:30:00
195905,3.43,4.17,4.81,3958.0,-54.0,49.0,60.0,4.17,2021-12-02 06:45:00


In [20]:
df_WarnowChacko_date_group = df_WarnowChacko_date.groupby('new_date').agg('mean').reset_index()
df_WarnowChacko_date_group

,new_date,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3
0,2021-08-23 19:00:00,12.050000,18.035000,18.880000,7672.0,-67.500000,94.000000,45.000000,18.035000
1,2021-08-23 19:15:00,12.226250,17.927500,18.696250,7684.0,-67.250000,94.000000,45.000000,17.927500
2,2021-08-23 19:30:00,11.734286,17.234286,17.677143,7699.0,-66.714286,92.571429,45.000000,17.234286
3,2021-08-23 19:45:00,11.942500,17.366250,18.236250,7714.0,-68.125000,93.000000,45.000000,17.366250
4,2021-08-23 20:00:00,13.081429,19.161429,19.652857,7729.0,-68.714286,93.000000,45.142857,19.161429
...,...,...,...,...,...,...,...,...,...
7146,2021-12-02 05:45:00,2.327143,4.022857,4.617143,21705.0,-66.714286,53.000000,50.142857,4.022857
7147,2021-12-02 06:00:00,2.636250,4.371250,4.682500,21720.0,-67.125000,53.000000,50.625000,4.371250
7148,2021-12-02 06:15:00,2.544286,4.455714,5.117143,21735.0,-67.142857,53.000000,51.000000,4.455714
7149,2021-12-02 06:30:00,2.350000,3.612500,4.002500,21750.0,-66.750000,52.625000,51.000000,3.612500


In [21]:
df_Urbana_date_group = df_Urbana_date.groupby('new_date').agg('mean').reset_index()
df_Urbana_date_group

,new_date,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3
0,2021-08-23 19:00:00,12.492500,16.487500,17.147500,5712.0,-58.250000,93.500000,44.500000,16.487500
1,2021-08-23 19:15:00,12.802500,16.617500,17.025000,5724.0,-58.750000,93.000000,43.750000,16.617500
2,2021-08-23 19:30:00,13.460000,17.500000,18.172857,5739.0,-59.571429,92.142857,45.142857,17.500000
3,2021-08-23 19:45:00,14.123750,18.346250,18.908750,5754.0,-58.625000,91.375000,46.500000,18.346250
4,2021-08-23 20:00:00,14.750000,19.221429,19.768571,5769.0,-57.428571,90.714286,50.000000,19.221429
...,...,...,...,...,...,...,...,...,...
9630,2021-12-02 05:45:00,3.647143,4.824286,5.487143,3900.0,-55.571429,50.000000,60.000000,4.824286
9631,2021-12-02 06:00:00,3.705000,4.990000,5.516250,3915.0,-56.625000,50.000000,60.000000,4.990000
9632,2021-12-02 06:15:00,3.728571,4.900000,5.474286,3930.0,-55.714286,49.142857,60.000000,4.900000
9633,2021-12-02 06:30:00,3.461250,4.751250,5.366250,3945.0,-55.000000,49.000000,60.000000,4.751250


In [22]:
df_UrbanaOnHolmes_date_group = df_UrbanaOnHolmes_date.groupby('new_date').agg('mean').reset_index()
df_UrbanaOnHolmes_date_group

,new_date,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3
0,2021-08-23 19:00:00,12.492500,16.487500,17.147500,5712.0,-58.250000,93.500000,44.500000,16.487500
1,2021-08-23 19:15:00,12.802500,16.617500,17.025000,5724.0,-58.750000,93.000000,43.750000,16.617500
2,2021-08-23 19:30:00,13.460000,17.500000,18.172857,5739.0,-59.571429,92.142857,45.142857,17.500000
3,2021-08-23 19:45:00,14.123750,18.346250,18.908750,5754.0,-58.625000,91.375000,46.500000,18.346250
4,2021-08-23 20:00:00,14.750000,19.221429,19.768571,5769.0,-57.428571,90.714286,50.000000,19.221429
...,...,...,...,...,...,...,...,...,...
9630,2021-12-02 05:45:00,3.647143,4.824286,5.487143,3900.0,-55.571429,50.000000,60.000000,4.824286
9631,2021-12-02 06:00:00,3.705000,4.990000,5.516250,3915.0,-56.625000,50.000000,60.000000,4.990000
9632,2021-12-02 06:15:00,3.728571,4.900000,5.474286,3930.0,-55.714286,49.142857,60.000000,4.900000
9633,2021-12-02 06:30:00,3.461250,4.751250,5.366250,3945.0,-55.000000,49.000000,60.000000,4.751250


In [23]:
df_WarnowChacko_date_group.to_feather('export_WarnowChacko_feather')
df_Urbana_date_group.to_feather('export_Urbana_feather')
df_UrbanaOnHolmes_date_group.to_feather('export_UrbanaOnHolmes_feather')